## Generate MARCXML based on the Library of Congress BIBFRAME2MARC mapping

In [1]:
%pip install rdflib
%pip install lxml
%pip install requests
%pip install saxonche


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Replace the URI with your Instance URI from Sinopia


In [2]:
# replace uri with the Instance uri of the resource you want to convert
uri = "https://api.stage.sinopia.io/resource/c3905dd8-e349-49b4-9ad3-55baeae178eb"

from saxonche import PySaxonProcessor
import lxml.etree as ET
from marc_xml.lc_bfxml_work import lc_bfxml_work, remove_last_non_blank_line
from marc_xml.lc_bfxml_instance import lc_bfxml_instance, remove_rdf_header


## Generate a combined LoC RDF Work and Instance file

In [4]:
lc_bfxml_work(uri)  
remove_last_non_blank_line() 
lc_bfxml_instance(uri) 
remove_rdf_header() 

# combine work first, then instance
with open("lc_bfxml_work.xml", "r") as work_file:
    work = work_file.read()
with open("lc_bfxml_instance.xml", "r") as instance_file:
    instance = instance_file.read()

#save as a file
with open("LoC_Work_Instance.xml", "w") as combined_file:
    combined_file.write(work + instance)

# add the sinopiabf namespace to the combined file
with open("LoC_Work_Instance.xml", "r") as file:
    filedata = file.read()
filedata = filedata.replace('<rdf:RDF', '<rdf:RDF xmlns:sinopiabf="http://sinopia.io/vocabulary/bf/"')
with open("LoC_Work_Instance.xml", "w") as file:
    file.write(filedata)

# apply "pre-transform-normalize.xsl" for normalization
dom = ET.parse("LoC_Work_Instance.xml")
xslt = ET.parse("marc_xml/xsl/pre-transform-normalize.xsl")
transform = ET.XSLT(xslt)
newdom = transform(dom)
with open("LoC_Work_Instance_Normalized.xml", "w") as f:
    f.write(str(newdom))    

print(newdom)


<rdf:RDF xmlns:bf="http://id.loc.gov/ontologies/bibframe/" xmlns:bflc="http://id.loc.gov/ontologies/bflc/" xmlns:sinopia="http://sinopia.io/vocabulary/" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
  <bf:Work rdf:about="https://api.stage.sinopia.io/resource/085754a9-082b-476e-84b3-137425e3226a">
    <sinopia:hasResourceTemplate>pcc:bf2:Monograph:Work</sinopia:hasResourceTemplate>
    <bf:title>
      <bf:Title rdf:nodeID="f4018806ecfd74df7bc6814f374be8e87b1">
        <bf:mainTitle xml:lang="en">Testing relations in Alma</bf:mainTitle>
      </bf:Title>
    </bf:title>
    <bf:contribution>
      <bf:PrimaryContribution rdf:nodeID="f4018806ecfd74df7bc6814f374be8e87b2">
        <bf:agent rdf:resource="http://id.loc.gov/authorities/names/no90020704"/>
      </bf:PrimaryContribution>
    </bf:contribution>
    <bf:originDate xml:lang="en">1948</bf:originDate>
    <bf:language>
      <bf:Language rdf:about="http://id.loc.gov/voc

## Generate MARCXML from the combined RDF/XML

In [5]:
# use PySaxonProcessor to appy bibframe2marc.xsl to LoC_Work_Instance_Normalized.xml
with PySaxonProcessor(license=False) as proc:
    xsltproc = proc.new_xslt30_processor()
    document = proc.parse_xml(xml_file_name="LoC_Work_Instance_Normalized.xml")
    executable = xsltproc.compile_stylesheet(stylesheet_file="bibframe2marc.xsl")
    output = executable.transform_to_string(xdm_node=document)
    #save as a file
    with open("marc.xml", "w") as f:
        f.write(output)
    print(output)


<?xml version="1.0" encoding="UTF-8"?>
<marc:record xmlns:marc="http://www.loc.gov/MARC21/slim">
   <marc:leader xml:space="preserve">     nam a22      c 4500</marc:leader>
   <marc:controlfield xml:space="preserve" tag="001">d0e1</marc:controlfield>
   <marc:controlfield xml:space="preserve" tag="003"/>
   <marc:controlfield xml:space="preserve" tag="007">t|</marc:controlfield>
   <marc:controlfield xml:space="preserve" tag="008">240208|||||||||is         |  ||| | heb  </marc:controlfield>
   <marc:datafield tag="035" ind1=" " ind2=" ">
      <marc:subfield code="a">1234567</marc:subfield>
   </marc:datafield>
   <marc:datafield tag="040" ind1=" " ind2=" ">
      <marc:subfield code="a">iljeel</marc:subfield>
      <marc:subfield code="b">eng</marc:subfield>
      <marc:subfield code="e">rda</marc:subfield>
      <marc:subfield code="c">iljeel</marc:subfield>
   </marc:datafield>
   <marc:datafield tag="100" ind1="1" ind2=" ">
      <marc:subfield code="a">Doe, John,</marc:subfield>
 